In [13]:
import MeCab 
import neologdn
from collections import defaultdict, namedtuple
tagger = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

NOUN_POS_ONLY = ('名詞',)
ALL_POS = ('名詞', '動詞', '形容詞', '接頭詞', '助動詞')
NOT_ADD_NOUN_POS = ('名詞', '動詞')
NOUN_POS_SUB = ('非自立',)
STOP_POS = ('記号', 'フィラー',)
NEW_WORD = '固有名詞'

def _get_new_word(token):
    if token.reading == '' and token.phonetic == '' and token.pos_detail1 == NEW_WORD and len(token.surface) > 1:
        return token.surface
    return None

def _filter_by_pos(sent, pos):
    base_forms = []
    new_words = []
    for token in _tokenize(sent):
        if token.pos in pos:
            if token.base_form != '*':
                base_forms.append(token.base_form)
            else:
                base_forms.append(token.surface)
            new_word = _get_new_word(token)
            if new_word is not None:
                new_words.append(new_word)
    return base_forms, new_words

def _tokenize(text):
    chunks = tagger.parse(text.rstrip()).splitlines()[:-1]  # Skip EOS
    token = namedtuple('Token', 'surface, pos, pos_detail1, pos_detail2, pos_detail3, '
                                'infl_type, infl_form, base_form, reading, phonetic')
    for chunk in chunks:
        if chunk == '':
            continue
        try:
            surface, feature = chunk.split('\t')
            feature = feature.split(',')
            if len(feature) <= 7:  # 読みがない
                feature.append('')
            if len(feature) <= 8:  # 発音がない
                feature.append('')
            yield token(surface, *feature)
        except:
            print('Exception occurred')

def replace_with_regex(text, remove_mention=True, remove_url=True):
    text = text.lower()
    replaced_text = re.sub(r'[【】]', ' ', text)  # 【】の除去
    replaced_text = re.sub(r'[・_!！？?☛]', '', replaced_text)  # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)  # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)  # ［］の除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'[⑤⑥②①③④⑦⑧⑨⑩]', '', replaced_text)
    replaced_text = re.sub(r'\d+', '', replaced_text)  # 数字の除去
    replaced_text = re.sub(r'[/。,、.=]', ' ', replaced_text)  # others
    replaced_text = re.sub(r'[●■]', '', replaced_text)
    if remove_mention:
        replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    if remove_url:
        replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    return replaced_text

In [14]:
text = 'BMWのレンタカー利用時に事故や車両の故障・不具合が発生した場合はどこに連絡すればいいですか？'

In [19]:
words = _filter_by_pos(replace_with_regex(neologdn.normalize(text)), ALL_POS)[0]

In [29]:
words

['BMW',
 'レンタカー',
 '利用',
 '時',
 '事故',
 '車両',
 '故障',
 '不具合',
 '発生',
 'する',
 'た',
 '場合',
 'どこ',
 '連絡',
 'する',
 'いい',
 'です']

In [7]:
import pandas as pd
merged = pd.read_csv('/home/ifte/resource/docomo_dic/data/merged_corpus.csv')

In [24]:
collec_titles = []
for title in merged['title'].values:
    collec_titles.append(_filter_by_pos(replace_with_regex(neologdn.normalize(title)), ALL_POS)[0])
    

In [ ]:
collec_titles

In [26]:
res = []
for items in collec_titles:
    x = set(words).intersection(items)
    if x:
        res.append(x)

In [32]:
sorted(res, key=lambda x: len(x), reverse=True)

[{'BMW',
  'いい',
  'する',
  'た',
  'です',
  'どこ',
  'レンタカー',
  '不具合',
  '事故',
  '利用',
  '場合',
  '故障',
  '時',
  '発生',
  '車両',
  '連絡'},
 {'いい', 'する', 'た', 'です', '事故', '利用', '場合', '時', '発生'},
 {'いい', 'する', 'です', 'どこ', 'レンタカー', '利用', '場合', '連絡'},
 {'いい', 'する', 'た', 'です', 'レンタカー', '利用', '場合', '車両'},
 {'BMW', 'いい', 'する', 'た', 'です', 'レンタカー', '利用', '場合'},
 {'BMW', 'いい', 'する', 'た', 'です', 'レンタカー', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合', '時'},
 {'いい', 'する', 'です', 'どこ', '利用', '場合', '連絡'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', 'レンタカー', '場合'},
 {'BMW', 'いい', 'する', 'た', 'です', 'レンタカー'},
 {'BMW', 'いい', 'する', 'た', 'です', 'レンタカー'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい', 'する', 'た', 'です', '利用', '場合'},
 {'いい

In [36]:
idx

[(1, 3), (2, 4)]

In [4]:
import re
s1 = 'nnBlue Berries'
pattern = 'Blu'
for match in re.finditer(pattern, s1):
    s = match.start()
    e = match.end()
    print('String match "%s" at %d:%d' % (s1[s:e], s, e))

String match "Blu" at 2:5


In [38]:
a = [1,2, 4]
b = [2, 3, 45, 4,2,34,2]
set(b).intersection(a)
set(b)

{2, 3, 4, 34, 45}

In [17]:
import editdistance
editdistance.eval('banana', 'bahama')
editdistance.eval('smtih', 'abc')

5

In [1]:
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance
damerau_levenshtein_distance('smtih', 'smith')  # expected result: 1
normalized_damerau_levenshtein_distance('smtih', 'smith')  # expected result: 0.2
damerau_levenshtein_distance([1, 2, 3, 4, 5, 6], [7, 8, 9, 7, 10, 11, 4])  # expected result: 7

from pyxdameraulevenshtein import damerau_levenshtein_distance_ndarray, normalized_damerau_levenshtein_distance_ndarray
import numpy as np
array = np.array(['test1', 'test12', 'test123'])
damerau_levenshtein_distance_ndarray('test', array)  # expected result: [1, 2, 3]
normalized_damerau_levenshtein_distance_ndarray('test', array)  # expected result: [0.2, 0.33333334, 0.42857143]

array([0.2       , 0.33333334, 0.42857143], dtype=float32)

In [6]:
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance
normalized_damerau_levenshtein_distance('smtih', 'abc')  # expected result: 0.2


1.0

In [14]:
normalized_damerau_levenshtein_distance([1, 2, 3, 4, 5, 6], [2, 3 , 5, 12, 100, 1, 2, 3, 5, 4, 6])

0.5454545617103577

In [39]:
import diff_match_patch as dmp_module

dmp = dmp_module.diff_match_patch()
diff = dmp.diff_main("apple", "appel")

In [45]:
collect = []
x = [collect.append(item) for key, item in diff if key==0]


In [57]:
xyz = [[519, 'BMWのレンタカー利用時に事故や車両の故障・不具合が発生した場合はどこに連絡すればいいですか？', 149.9806202716682], [282, 'どこに連絡すればいいですか？', 49.110381953177196], [501, 'どこに連絡すればいいですか？', 31.259098843380567], [326, 'すればいいですか？', 88.57281705118818], [284, 'すればいいですか？', 85.93458781308132], [456, 'すればいいですか？', 71.1931094355778], [350, 'すればいいですか？', 70.42370602787592], [264, 'すればいいですか？', 69.54996427301445], [287, 'BMWのレンタカー', 64.74019941473117], [286, 'BMWのレンタカー', 64.13605774152263], [289, 'BMWのレンタカー', 63.10493308569892], [518, 'BMWのレンタカー', 62.98078722761835], [319, 'BMWのレンタカー', 56.880497904992595], [470, 'すればいいですか？', 56.8193204470759], [487, 'すればいいですか？', 56.670677465918814], [288, 'BMWのレンタカー', 54.89539239623433], [309, 'すればいいですか？', 52.638993731130974], [327, 'すればいいですか？', 51.83145831773621], [305, 'BMWのレンタカー', 50.993935937033925], [320, 'BMWのレンタカー', 48.88456047441369], [468, 'すればいいですか？', 46.79050646071987], [300, 'BMWのレンタカー', 46.75940427726099], [298, 'BMWのレンタカー', 46.03774060688882], [297, 'BMWのレンタカー', 45.862298045643314], [323, 'BMWのレンタカー', 45.61863675911687], [303, 'BMWのレンタカー', 45.482841089591126], [304, 'BMWのレンタカー', 44.78395149872963], [587, 'すればいいですか？', 44.67834180850602], [301, 'BMWのレンタカー', 44.55780264483805], [299, 'BMWのレンタカー', 43.83517630123558], [322, 'BMWのレンタカー', 43.55619013044913], [321, 'BMWのレンタカー', 43.35421781009619], [307, 'BMWのレンタカー', 43.096028181387595], [281, 'すればいいですか？', 42.24396902756773], [496, 'すればいいですか？', 41.29588503548163], [411, 'すればいいですか？', 39.57394451428246], [353, 'すればいいですか？', 39.54782564119905], [342, 'すればいいですか？', 39.142957186428774], [461, 'すればいいですか？', 39.044898810683826], [308, 'BMWのレンタカー', 38.734358215900826], [406, 'すればいいですか？', 37.952890579895964], [464, 'すればいいですか？', 37.85899173025], [349, 'すればいいですか？', 37.49120028259084], [492, 'すればいいですか？', 37.49120028259084], [1062, 'すればいいですか？', 32.61843737189797], [495, 'すればいいですか？', 30.873118234933038], [573, 'すればいいですか？', 30.246658451502583], [584, 'すればいいですか？', 29.798659478661325], [345, 'すればいいですか？', 29.1586808366957], [407, 'すればいいですか？', 29.1586808366957], [493, 'すればいいですか？', 29.1586808366957], [463, 'すればいいですか？', 29.08190312899357], [503, 'すればいいですか？', 29.02264107253388], [354, 'すればいいですか？', 28.807081582007406], [344, 'すればいいですか？', 28.112741777320892], [409, 'すればいいですか？', 28.112741777320892], [491, 'すればいいですか？', 28.112741777320892], [1061, 'すればいいですか？', 28.036712855567835], [458, 'すればいいですか？', 27.745203970714375], [325, 'すればいいですか？', 27.151083919615637], [347, 'すればいいですか？', 25.850610064689654], [410, 'すればいいですか？', 25.850610064689654], [494, 'すればいいですか？', 25.850610064689654], [460, 'すればいいですか？', 25.775075596112504], [574, 'すればいいですか？', 25.77479346149773], [581, 'すればいいですか？', 25.77479346149773], [596, 'すればいいですか？', 25.77479346149773], [405, 'すればいいですか？', 25.65061570256618], [346, 'すればいいですか？', 25.633568757796375], [582, 'すればいいですか？', 25.633568757796375], [583, 'すればいいですか？', 25.633568757796375], [462, 'すればいいですか？', 25.626207076036486], [457, 'すればいいですか？', 25.480198566765292], [408, 'すればいいですか？', 24.468215271129075], [348, 'すればいいですか？', 24.405528327498274], [489, 'すればいいですか？', 24.405528327498274], [607, 'すればいいですか？', 23.80775037841483], [504, 'すればいいですか？', 23.638106559804616], [397, 'すればいいですか？', 20.83879593163166], [271, 'すればいいですか？', 20.730152428636185], [428, 'すればいいですか？', 20.730152428636185], [451, 'すればいいですか？', 20.676501908672698], [598, 'すればいいですか？', 20.420051206718007], [276, 'すればいいですか？', 20.302499231631852], [1060, 'すればいいですか？', 20.302499231631852], [431, 'すればいいですか？', 19.874311391268606], [205, 'すればいいですか？', 19.614662909586464], [171, 'すればいいですか？', 19.009611433508237], [488, 'すればいいですか？', 19.001236424537222], [351, 'すればいいですか', 27.613760032181837], [285, 'レンタカー利用', 24.546344968778946], [469, 'レンタカー利用', 24.380056290133528], [667, 'レンタカー利用', 23.654870743096865], [293, 'いいですか？', 43.9747068264399], [239, 'いいですか？', 43.763118141793214], [530, 'いいですか？', 43.763118141793214], [365, 'いいですか？', 31.495598015119526], [246, 'いいですか？', 31.320135625858853], [459, 'すればいいで', 28.50280454985475], [341, 'すればいいで', 26.0039397293]]

In [ ]:
merged_corpus = pd.read_csv(os.path.join(kwargs['train_data'].data_dir, MERGED_CORPUS))

distance_collect = []
# for ids, score in best_idx_scores[:300]:
#     distance_collect.append([ids, set(q).intersection(tokenize(merged_corpus.iloc[ids].title)), score])
# print(distance_collect)
dmp = dmp_module.diff_match_patch()
            
for ids, score in best_idx_scores[:300]:
    collect = []
    match = dmp.diff_main(message.query, merged_corpus.iloc[ids].title)
    [collect.append(item) for key, item in match if key == 0]
    if collect:
        distance_collect.append([ids, sorted(collect, key=len)[-1], score])

best_idx_scores = sorted(distance_collect, key=(lambda tup: len(tup[1])), reverse=True)
if len(message.query) - len(best_idx_scores[0][1]) < 3:
    # print(best_idx_scores)
    idx = []
    [idx.append((x, z)) for x, y, z in best_idx_scores]
    best_idx_scores = idx

In [2]:
'xyz asf fffwsff'.find('wss')

-1